In [6]:
import json

In [8]:
# load json file to extend the node info
with open("../data/aggregated_data.json") as jr:
    cve_json = json.load(jr)



In [18]:
order = 0
for key, value in cve_json.items():
    while order >= 10:
        print(key)
        print(value)
        order += 1
        

In [15]:
list(cve_json.keys())[:20]

['org.jenkins-ci.main:cli:1.591',
 'org.elasticsearch:elasticsearch-hadoop:2.2.0-beta1',
 'org.jvnet.hudson.main:hudson-core:1.128',
 'net.sourceforge.htmlunit:neko-htmlunit:2.47.0',
 'org.jenkins-ci.main:cli:1.592',
 'org.jvnet.hudson.main:hudson-core:1.129',
 'net.sourceforge.htmlunit:neko-htmlunit:2.47.1',
 'org.springframework.boot:spring-boot-actuator:1.2.6.RELEASE',
 'org.jenkins-ci.main:cli:1.593',
 'org.jenkins-ci.main:cli:1.594',
 'org.jvnet.hudson.main:hudson-core:1.124',
 'org.xwiki.rendering:xwiki-rendering-syntax-xhtml:7.3-milestone-1',
 'org.jvnet.hudson.main:hudson-core:1.125',
 'org.jvnet.hudson.main:hudson-core:1.126',
 'org.jenkins-ci.main:cli:1.590',
 'org.jvnet.hudson.main:hudson-core:1.127',
 'org.jvnet.hudson.main:hudson-core:1.120',
 'org.jvnet.hudson.main:hudson-core:1.121',
 'org.jvnet.hudson.main:hudson-core:1.122',
 'org.jvnet.hudson.main:hudson-core:1.123']

In [12]:
len(list(cve_json.keys()))

97054